In [1]:
import pandas as pd
import numpy as np
import copy
import json
from Algorithm import ProvenanceSearchValues_8_complex_optimized_binary as ps
from Algorithm import LatticeTraversal_5_20230121 as lt


In [2]:
data = pd.read_csv(r"../data/uniform.csv", sep=' ')
print(len(data))
data.head()

10000


,x,y,blue
0,196.117936,290.275447,0
1,748.658545,683.729258,0
2,280.366373,507.097135,1
3,238.944206,72.706203,1
4,953.546180,190.831150,1


In [69]:
# data = data[['blue', 'x', 'y']]
# data.to_csv(r"../data/uniform.csv", sep=' ', index=False)


In [7]:
sorted_data = data.sort_values(by = ['x', 'y'], inplace=False)
sorted_data

,x,y,blue
3323,0.119246,33.260516,0
7373,0.178170,842.909829,0
5501,0.283920,306.770076,0
8005,0.413244,580.062109,1
6262,0.568892,256.354380,1
...,...,...,...
5934,999.519248,166.353209,0
9000,999.571220,99.813674,1
5960,999.580062,271.428169,0
8801,999.634059,66.948577,1


In [13]:
sorted_data[80:140]

,x,y,blue
9504,9.084502,489.981738,0
6877,9.189022,274.166467,1
6302,9.428817,173.372911,1
2595,9.748168,118.935805,1
5665,9.830845,516.236945,0
629,9.835156,331.106346,0
2870,9.902150,619.214686,0
3362,9.947147,494.370034,0
5320,9.996766,454.935278,0
6281,10.116066,37.242826,0


In [56]:
ydata = data[(data['y'] >= 20) & (data['y'] <= 40)]
print(len(ydata))

200


In [57]:
xdata = ydata[(ydata['x'] >= 10) & (ydata['x'] <= 30)]
print(len(xdata))

3


In [22]:
import pandas as pd

# Assuming you have a dataframe named 'dataframe' with only one column 'column_name'
# Replace 'column_name' with the actual column name in your dataframe
# Sample data for demonstration purposes
data = {'column_name': list(range(1000))}  # Creating a column with 1000 values (0 to 999)
dataframe = pd.DataFrame(data)

# Original dataframe
print("Original DataFrame:")
print(dataframe)

# Dropping the last 900 values from 'column_name'
dataframe = dataframe[:-900]

# Updated dataframe with only the first 100 values in 'column_name'
print("\nDataFrame with last 900 values dropped:")
print(dataframe)


Original DataFrame:
     column_name
0              0
1              1
2              2
3              3
4              4
..           ...
995          995
996          996
997          997
998          998
999          999

[1000 rows x 1 columns]

DataFrame with last 900 values dropped:
    column_name
0             0
1             1
2             2
3             3
4             4
..          ...
95           95
96           96
97           97
98           98
99           99

[100 rows x 1 columns]


# analyze result refinement

In [5]:
def FindMinimalRefinement(data_file_prefix, separator, query_file, constraint_file, data_file_format,
                          single_binary_sensitive_att, time_limit=5 * 60):
    global assign_to_provenance_num
    assign_to_provenance_num = 0
    # data = pd.read_csv(data_file, index_col=False)
    with open(query_file) as f:
        query_info = json.load(f)
    tables = query_info['tables']
    joinkeys = []
    comparekeys = []
    if "joinkeys" in query_info:
        joinkeys = query_info["joinkeys"]
    if "comparekeys" in query_info:
        comparekeys = query_info["comparekeys"]
    numeric_attributes = []
    categorical_attributes = {}
    selection_numeric_attributes = {}
    selection_categorical_attributes = {}
    if 'selection_numeric_attributes' in query_info:
        selection_numeric_attributes = query_info['selection_numeric_attributes']
        numeric_attributes = list(selection_numeric_attributes.keys())
    if 'selection_categorical_attributes' in query_info:
        selection_categorical_attributes = query_info['selection_categorical_attributes']
        categorical_attributes = list(selection_categorical_attributes.keys())  # query_info['categorical_attributes']
    selected_attributes = numeric_attributes + [x for x in categorical_attributes]
    # print("selected_attributes", selected_attributes)

    with open(constraint_file) as f:
        constraint_info = json.load(f)

    sensitive_attributes = constraint_info['all_sensitive_attributes']
    fairness_constraints = constraint_info['fairness_constraints']

    pd.set_option('display.float_format', '{:.2f}'.format)
    assign_to_provenance_num += 1
    # data:after join
    whether_satisfy, data = ps.whether_satisfy_fairness_constraints(data_file_prefix, separator, data_file_format, tables,
                                                                 joinkeys, comparekeys, selected_attributes,
                                                                 sensitive_attributes, fairness_constraints,
                                                                 numeric_attributes, categorical_attributes,
                                                                 selection_numeric_attributes,
                                                                 selection_categorical_attributes)


    # whether it's single-direction
    only_smaller_than = True
    only_greater_than = True
    for fc in fairness_constraints:
        if 'first_sensitive_attributes' in fc:
            only_greater_than = False
            only_smaller_than = False
            break
        elif fc['symbol'] == ">=" or fc['symbol'] == ">":
            only_smaller_than = False
        else:
            only_greater_than = False
        if (not only_greater_than) and (not only_smaller_than):
            break
    fairness_constraints_provenance_greater_than, fairness_constraints_provenance_smaller_than, \
        fairness_constraints_provenance_complex, contraction_threshold, data \
        = ps.subtract_provenance_refinement(data, selected_attributes, sensitive_attributes,
                                         fairness_constraints,
                                         numeric_attributes, categorical_attributes,
                                         selection_numeric_attributes,
                                         selection_categorical_attributes, only_greater_than,
                                         only_smaller_than)

    PVT, PVT_head, categorical_att_columns, \
        max_index_PVT, possible_values_lists = ps.build_PVT_refinement(data, selected_attributes,
                                                                    numeric_attributes,
                                                                    categorical_attributes,
                                                                    selection_numeric_attributes,
                                                                    selection_categorical_attributes,
                                                                    sensitive_attributes,
                                                                    fairness_constraints,
                                                                    fairness_constraints_provenance_greater_than,
                                                                    fairness_constraints_provenance_smaller_than,
                                                                    fairness_constraints_provenance_complex,
                                                                    contraction_threshold)

    return PVT


In [38]:
temp_data = copy.deepcopy(data)

data_file_prefix = r"../data/"
query_file_prefix = r"./q"
constraint_file_prefix = r"./"
time_limit = 60 * 30
separator = ' '

time_output_prefix = r"./result_"

def run(c, q):
    constraint_file = r"./constraint_" + c + ".json"

    time_output_file = r"./query_change_q1_" + c + ".csv"
    time_output = open(time_output_file, "w")
    time_output.write("_,PS,PS_prov,PS_search,base,base_prov,base_search\n")

    result_output_file = r"./result_query_change_q1" + c + ".txt"
    result_output = open(result_output_file, "w")
    result_output.write("selection file, result\n")

    data_format = ".csv"
    print("query", q)
    query_file = query_file_prefix + str(q) + ".json"
    single_binary_sensitive_att = True

    PVT = FindMinimalRefinement(data_file_prefix, separator, query_file, constraint_file, data_format, single_binary_sensitive_att, time_limit)
    return PVT



def analyze_x_smaller(temp_data, PVT, x_smaller_idx):
    x_smaller = PVT['x__smaller'].tolist()[x_smaller_idx]
    temp_data = temp_data[temp_data['x'] < x_smaller]
    blue_num = len(temp_data[temp_data['blue'] == 1])
    red_num = len(temp_data[temp_data['blue'] == 0])
    print("x idx = {}, x_smaller_val = {}, blue = {}, red = {}, abs = {}".format(x_smaller_idx, x_smaller, blue_num, red_num, (blue_num - red_num)))


def analyze_y_smaller(temp_data, PVT, y_smaller_idx):
    y_smaller = PVT['y__smaller'].tolist()[y_smaller_idx]
    temp_data = temp_data[temp_data['y'] < y_smaller]
    blue_num = len(temp_data[temp_data['blue'] == 1])
    red_num = len(temp_data[temp_data['blue'] == 0])
    print("y idx = {}, y_smaller_val = {}, blue = {}, red = {}, abs = {}".format(y_smaller_idx, y_smaller, blue_num, red_num, (blue_num - red_num)))


def analyze_y_greater(temp_data, PVT, y_greater_idx):
    y_greater = PVT['y__greater'].tolist()[y_greater_idx]
    temp_data = temp_data[temp_data['y'] >= y_greater]
    blue_num = len(temp_data[temp_data['blue'] == 1])
    red_num = len(temp_data[temp_data['blue'] == 0])
    row = temp_data[temp_data['y'] == y_greater]
    print("y idx = {}, y_greater_val = {}, blue = {}, red = {}, abs = {}".format(y_greater_idx, y_greater, blue_num, red_num, (blue_num - red_num)))
    row.reset_index(drop=True, inplace=True)
    if len(row) > 0:
        print(row.iloc[0, :].tolist())



In [39]:
PVT = run('refine21', 2)
y_smaller_idx = 0
y_smaller = PVT['y__smaller'].tolist()[y_smaller_idx]
temp_data = temp_data[temp_data['y'] <= y_smaller]
for y_greater_idx in range(140, -1, -1):
    analyze_y_greater(temp_data, PVT, y_greater_idx)

query 2
10000
prepare time = 0.0006120204925537109
10000
PVT_head: []
y idx = 140, y_greater_val = 407.0165497619304, blue = 1003, red = 969, abs = 34
[989.6768257328298, 407.0165497619304, 0.0]
y idx = 139, y_greater_val = 407.0040835979928, blue = 1004, red = 969, abs = 35
[338.8749315271975, 407.0040835979928, 1.0]
y idx = 138, y_greater_val = 393.12333704447576, blue = 1069, red = 1042, abs = 27
[816.7244800480705, 393.12333704447576, 1.0]
y idx = 137, y_greater_val = 406.8765072624598, blue = 1004, red = 970, abs = 34
[299.40660276349394, 406.8765072624598, 0.0]
y idx = 136, y_greater_val = 406.87241139661927, blue = 1004, red = 971, abs = 33
[893.0314567127825, 406.87241139661927, 0.0]
y idx = 135, y_greater_val = 406.8364947587475, blue = 1005, red = 971, abs = 34
[918.0804476804984, 406.8364947587475, 1.0]
y idx = 134, y_greater_val = 406.8213966904883, blue = 1006, red = 971, abs = 35
[190.54191595589447, 406.8213966904883, 1.0]
y idx = 133, y_greater_val = 393.37000391244527,

In [74]:
temp_data = data[(data['x'] >= 100) & (data['x'] <= 450) ]
print(len(temp_data))
temp_data = data[(data['y'] >= 450) & (data['y'] <= 600)]
print(len(temp_data))

temp_data = data[(data['x'] >= 100) & (data['x'] <= 250)  & (data['y'] >= 400) & (data['y'] <= 600)]
print(len(temp_data))

blue_num = len(temp_data[temp_data['blue'] == 1])
red_num = len(temp_data[temp_data['blue'] == 0])
print(blue_num, red_num, blue_num - red_num)


3469
1535
312
169 143 26
